<a href="https://colab.research.google.com/github/lonelu/Rvrsprot/blob/main/Colab_Rvrsprot_master_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rvrsprot_master_loop_colab
DeGrado Lab, UCSF.

The program is used for loop selections from Master result. 

In [1]:
#@title Upload pdb
import os
from google.colab import files
import re
import datetime
import shutil
#@markdown Define job
jobname = 'None' #@param {type:"string"}

#@markdown Upload pdb prompt
uploaded = files.upload()
for fn in uploaded.keys():
  # remove any preceding '.'s in filename
  count = fn.count(".") - 1
  newfn = fn.replace('.', '-', count)
  newfn = fn.replace('_', '-')
pdbname = newfn.split('.')[0]
shutil.copyfile(fn, pdbname + '.pdb')



Saving 00009.f63440efff7e.allbb_ala_min_ala_0001.pdb to 00009.f63440efff7e.allbb_ala_min_ala_0001 (2).pdb


'00009.pdb'

In [2]:
#@title Set parameters
import os
from google.colab import files
import re
import datetime

if jobname == 'None':
  jobname = pdbname

time_tag = datetime.datetime.now().strftime('%Y%m%d-%H%M%S') 

workdir = '/content/' + jobname + '_' + time_tag 
if not os.path.exists(workdir):
  os.mkdir(workdir)

#@markdown Set parameter

loop_range_min = 11 #@param {type:"integer"}
loop_range_max = 13 #@param {type:"integer"}

loop_target_list = 'master_origin' #@param {type:"string"}
rmsdCut = 1.5 #@param {type:"number"} 

master_query_loop_top = 500 #@param {type:"integer"}
cluster_rmsd = 1.0 #@param {type:"number"}
rm_duplicate = True #@param {type:"boolean"} 
#@markdown - (if the database have duplicated pdbs.)

loop_topo_sels_front = 'A-30-36-A-39-45, A-104-110-A-113-119' #@param {type:"string"}
loop_topo_sels_back = 'None' #@param {type:"string"}

cluster_count_cut = 10 #@param {type:"number"}
select_min_rmsd_pdb= True #@param {type:"boolean"} 
#@markdown - (output will copy the result pdb with min rmsd.)

class Para():
  workdir = workdir
  target_file = pdbname

  loop_topo_sels = {}

  if loop_topo_sels_front != 'None' and loop_topo_sels_back != '':
    xx = re.sub(r'\s+', '', loop_topo_sels_front)
    xx = xx.split(',')
    front = {}
    for x in xx:
      _xs = x.split('-')
      key = _xs[0] + '-' + _xs[3]
      if key in front.keys():
        front[key].append((_xs[0], int(_xs[1]), int(_xs[2]), _xs[3], int(_xs[4]), int(_xs[5])))
      else:
        front[key] = [(_xs[0], int(_xs[1]), int(_xs[2]), _xs[3], int(_xs[4]), int(_xs[5]))]
    loop_topo_sels['front'] = front

  if loop_topo_sels_back != 'None' and loop_topo_sels_back != '':
    xx = re.sub(r'\s+', '', loop_topo_sels_back)
    xx = xx.split(',')
    back = {}
    for x in xx:
      _xs = x.split('-')
      key = _xs[0] + '-' + _xs[3]
      if key in back.keys():
        back[key].append((_xs[0], int(_xs[1]), int(_xs[2]), _xs[3], int(_xs[4]), int(_xs[5])))
      else:
        back[key] = [(_xs[0], int(_xs[1]), int(_xs[2]), _xs[3], int(_xs[4]), int(_xs[5]))]
    loop_topo_sels['back'] = back   

  ### Master search para
  loop_range = [loop_range_min, loop_range_max]
  if loop_target_list == 'master_origin':
    loop_target_list = '/content/master_db/list'

  rmsdCut = rmsdCut
  master_query_loop_top = master_query_loop_top
  cluster_rmsd = cluster_rmsd
  rm_duplicate = rm_duplicate

  ### Output para
  cluster_count_cut = cluster_count_cut
  select_min_rmsd_pdb= select_min_rmsd_pdb



In [3]:
#@title Install Rvrsprot.

%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

%%bash -s

set -e

cd /content

rm -rf RVRSPROT_READY METALPROT_READY
rm -rf Rvrsprot Metalprot
if [ ! -f RVRSPROT_READY ]; then
  echo "Cloning Rvrsprot..."
  rm -rf Rvrsprot
  git clone https://github.com/lonelu/Rvrsprot.git
  cd /content/Rvrsprot
  pip install -e . &> /dev/null
  touch RVRSPROT_READY
fi

cd /content/

if [ ! -f METALPROT_READY ]; then
  echo "Cloning Metalprot..."
  rm -rf Metalprot
  git clone https://github.com/lonelu/Metalprot.git

  cd /content/Metalprot
  pip install -e . &> /dev/null
  touch METALPROT_READY
fi

chmod 777 /content/Rvrsprot/rvrsprot/external/createPDS
chmod 777 /content/Rvrsprot/rvrsprot/external/parsePDS
chmod 777 /content/Rvrsprot/rvrsprot/external/master


#download database and change list filepath.
pip install logomaker &> /dev/null
pip install fpdf &> /dev/null


<IPython.core.display.Javascript object>

In [7]:
#@title Download database

%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

%%bash -s



set -e
#rm -rf DB_READY DB_READY2 
#rm -rf /content/master_db
if [ ! -f DB_READY ]; then
  rm -rf /content/master_db.zip
  curl -L https://ucsf.box.com/shared/static/3wabh6hfii8fl3bui9txpox9l9rphygv.zip --output /content/master_db.zip 
  ls -l /content/master_db.zip
  touch DB_READY
fi

if [ ! -f DB_READY2 ]; then
  unzip -q /content/master_db.zip
  ls -l /content
  sed -i 's/\/wynton\/home\/degradolab\/lonelu\/DesignData\/Database/\/content/g' /content/master_db/list
  head /content/master_db/list
  touch DB_READY2
fi



<IPython.core.display.Javascript object>

In [9]:
#@title Run master loop.

%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

import os
import sys
sys.path.append(r'/content/Metalprot')
sys.path.append(r'/content/Rvrsprot')
from rvrsprot.loop import loop_ss
import datetime
import subprocess


para = Para()
def run():
    #para = Para()
    loop_ss.run_loop_ss(para.workdir + '/', '/content/' + para.target_file + '.pdb', para.loop_topo_sels, para)
    return

run()  


<IPython.core.display.Javascript object>

In [10]:
#@title Download results

!cd /content/
!zip -FSrq $para.workdir".zip" $para.workdir"/"
para = Para()
files.download(para.workdir + '.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The program is for DeGrado lab internal usage now (10/25/2022).

Please contact lonelur@gmail.com if you find something wrong or if you have special suggestions to help other researchers.